# Plant Health classification

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # This ensures the file has the correct permissions

In [ ]:
!kaggle datasets download -d rishirishabh/plant-health-dataset-for-cnn

In [ ]:
import zipfile

# Unzipping the downloaded dataset
zip_ref = zipfile.ZipFile('/content/plant-health-dataset-for-cnn.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerators for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% of data for validation
)

# Create training and validation datasets
train_ds = datagen.flow_from_directory(
    directory='/content/color',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Set as training data
)

validation_ds = datagen.flow_from_directory(
    directory='/content/color',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Set as validation data
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# Define image size
img_size = 256  # Replace with your image size if different

# Create CNN model
model = Sequential()

# Add layers
# First layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D(2, 2))

# Second layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(train_ds.num_classes, activation='softmax'))  # Adjust if binary classification

In [ ]:
model.summary()

In [ ]:
# Compile the Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Use sparse_categorical_crossentropy
              metrics=['accuracy'])

history = model.fit(train_ds , epochs = 5 , validation_data = validation_ds)

In [ ]:
# Save the entire model to a file
model.save('model.h5')  # You can choose a different filename if you prefer

In [ ]:
# Evaluate the model on validation dataset
val_loss, val_accuracy = model.evaluate(validation_ds)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Epochs vs Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
